# 06-01: Graphic Degree Sequences

*November 23 2022*  

In this notebook, we implement methods to check whether a number sequence is graphic, i.e. whether a graph exists such that the sequence is the degree sequence of the graph.

In [1]:
import pathpy as pp
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

plt.style.use('default')
sns.set_style("whitegrid")

# Detecting graphic degree sequences

We first implement a function to detect whether a degree sequence is graphic. For this we use results presented by Behzad and Chartrand (1967) and Erdös and Gallai (1960) as summarised [here](http://mathworld.wolfram.com/GraphicSequence.html).

In [2]:
def is_graphic_sequence(degree_sequence):
   
    S = sum(degree_sequence)
    n = len(degree_sequence)

    # necessary condition: sum of degrees must be even (which renders this method invalid if we allow self-loops that are counted as one in the degrees)
    if S%2 != 0:
        return False

    # necessary condition: Behzad and Chartrand (1967) showed that in a graphic degree
    # sequence at least one degree must occur twice (this only holds for networks 
    # without self_loops!). If no degree occurs twice we return False
    if len(set(degree_sequence)) == len(degree_sequence):
        return False

    # Sort degrees in descending order
    ordered_sequence = sorted(degree_sequence, reverse=True)

    # check necessary and sufficient condition shown by Erdös and Gallai (1960)
    # see http://mathworld.wolfram.com/GraphicSequence.html for details
    
    for r in range(1, n):
        
        # sum on left side
        S = 0
        for j in range(1, r+1):
            S += ordered_sequence[j-1]
            
        # sum on right side
        M = 0
        for i in range(r+1, n+1):
            M += min(r, ordered_sequence[i-1])
        # return False if condition is violated
        # for ANY r
        if S > r * (r-1) + M:
            return False

    return True

We now test our function for some sequences, for which we either know that they are graphic or not:

In [3]:
# In a network without self-loops, at least one degree has to occur twice. It is easy 
# to verify that no network (without self-loops) with the following degree sequence can exist
print('[2, 1, 3]:\t', is_graphic_sequence([2, 1, 3]))

# In a fully connected network with three nodes and no self-loops, each node has 
# degree 2, so this is necessarily a graphic sequence
print('[2, 2, 2]:\t', is_graphic_sequence([2, 2, 2]))

# This is not a graphic sequence (unless we allow self-loops)
print('[1]:\t\t', is_graphic_sequence([1]))

# This is not a graphic sequence (unless we allow self-loops and count their degree as two)
print('[0, 2]:\t\t', is_graphic_sequence([0, 2]))

# This is a graphic sequence
print('[1, 3, 1, 1]:\t', is_graphic_sequence([1, 3, 1, 1]))

# This is not a graphic sequence
print('[1, 3, 0, 2]:\t', is_graphic_sequence([1, 3, 0, 2]))

# This is a graphic sequence
print('[3, 2, 2, 1]:\t', is_graphic_sequence([3, 2, 2, 1]))

# This is a graphic sequence
print('[3, 2, 2, 1]:\t', is_graphic_sequence([3, 2, 2, 1]))

# This is not a graphic sequence
print('[3, 2, 2, 4]:\t', is_graphic_sequence([3, 2, 2, 4]))

[2, 1, 3]:	 False
[2, 2, 2]:	 True
[1]:		 False
[0, 2]:		 False
[1, 3, 1, 1]:	 True
[1, 3, 0, 2]:	 False
[3, 2, 2, 1]:	 True
[3, 2, 2, 1]:	 True
[3, 2, 2, 4]:	 False


Do we really have to sort the degrees in the above function? Let us create a function that is exactly the same as above, but where we do not impose the condition $d_1 \leq d_2 \leq \ldots$.

In [4]:
def is_graphic_sequence_wrong(degree_sequence, self_loops=False, multi_edges=False):
   
    assert not (not self_loops and multi_edges), 'Networks with multi_edges and no self_loops are not supported'
    S = sum(degree_sequence)
    n = len(degree_sequence)

    # the sum of degrees must always be even (assuming self-loops are counted with degree of two)
    if S%2 != 0:
        return False

    # if multi-edges are allowed we are done
    if multi_edges:
        return True

    # use Behzad and Chartrand 1967, which shows that in a graphic degree
    # sequence at least one degree must occur twice (holds for networks without self_loops)
    if not self_loops and len(set(degree_sequence)) == len(degree_sequence):
        return False

    # This is wrong: 
    ordered_sequence = degree_sequence # sorted(degree_sequence, reverse=True)

    # check necessary and sufficient condition given by Erdös and Gallai (1960)
    # see http://mathworld.wolfram.com/GraphicSequence.html

    for r in range(1, n):
        M = 0
        S = 0
        for i in range(0, r):
            S += ordered_sequence[i]
        for i in range(r, n):
            M += min(r+1, ordered_sequence[i])
        if S > r * (r+1) + M:
            return False

    return True

We find that for the following (non-graphic) degree sequence the updated function returns a wrong result:

In [5]:
sequence = [1, 1, 90]

print(str(sequence)+'\t', is_graphic_sequence(sequence))
print(str(sequence)+'\t', is_graphic_sequence_wrong(sequence))

[1, 1, 90]	 False
[1, 1, 90]	 True


`pathpy` provides a function that you can use to test whether a sequence is graphic or not. You can use it as follows:

In [6]:
pp.generators.is_graphic_Erdos_Gallai(sequence)

False